In [29]:
##imports
import os
import io
import time
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display
from ipywidgets import Textarea, FileUpload, Button, VBox, HTML

In [30]:
OLLAMA_BASE_URL = "https://localhost:11434/v1"
MODEL = "llama3.2"

In [31]:
openai = OpenAI(base_url = OLLAMA_BASE_URL, api_key="llama")

In [32]:
#Step 1 : Creating my own prompts:

system_prompt = "You are an assistant who analyzes user's CV against the job description \
 and provide a short summary if the user is fit for this job. If the user is fit for the job \
 write a cover letter for the user to apply for the job. Keep the cover letter professional, short, \
 and formal. \
 Important things to notice before analyzing CV:\
 1. Always check if the CV is actually a CV or just random text\
 2. Check if the job description fetched from the website is the job description or not\
     and ignore text related to navigation\
 3. Also check the link of the job posting, if it actually resembles a job posting or is just random \
     fake website\
 4. if any one of these two checks fails, do not analyze the CV against the Job description and give an\
     appropriate response as you think\
 5. Always respond in Markdown."

for_user_prompt = {
    'job_posting_url':'',
    'job_posting': '',
    'cv_text': ''
}

In [ ]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [34]:
# Create widgets - to create a box for the job posting text
job_posting_url_area = Textarea(
    placeholder='Paste the URL of the job posting here, ONLY URL PLEASE',
    description='Fetching job:',
    disabled=False,
    layout={'width': '800px', 'height': '50px'}
)

status_job_posting = HTML(value="<b>Status:</b> Waiting for inputs...")

# Create Submit Buttons
fetch_job_posting_button = Button(description='Fetch Job Posting', button_style='primary')

def fetch_job_posting_action(b):
    for_user_prompt['job_posting_url'] = job_posting_url_area.value
    if for_user_prompt['job_posting_url']:
        ed = Website(for_user_prompt['job_posting_url'])
        status_job_posting.value = "<b>Status:</b> Job posting fetched successfully!"
        fetch_job_posting_button.button_style='success'
        for_user_prompt['job_posting']=ed.text
    else:
        status_job_posting.value = "<b>Status:</b> Please enter a job posting url before submitting."

# Attach actions to buttons
fetch_job_posting_button.on_click(fetch_job_posting_action)

# Layout
job_posting_box = VBox([job_posting_url_area, fetch_job_posting_button])

# Display all widgets
display(VBox([
    HTML(value="<h2>Input Job Posting Url</h2>"),
    job_posting_box,
    status_job_posting
]))

In [35]:
!pip install PyPDF2


In [36]:
import PyPDF2

# Define file upload for CV
cv_upload = FileUpload(
    accept='.pdf',  # Only accept PDF files
    multiple=False,  # Only allow single file selection
    description='Upload CV (PDF)'
)

status = HTML(value="<b>Status:</b> Waiting for inputs...")

# Create Submit Buttons
submit_cv_button = Button(description='Submit CV', button_style='success')

# Functions
def submit_cv_action(change):

    if not for_user_prompt['cv_text']:
        status.value = "<b>Status:</b> Please upload a CV before submitting."
        
    if cv_upload.value:
        # Get the uploaded file
        uploaded_file = cv_upload.value[0]
        content = io.BytesIO(uploaded_file['content'])
        
        try:
            pdf_reader = PyPDF2.PdfReader(content) 
            cv_text = ""
            for page in pdf_reader.pages: 
                cv_text += page.extract_text() 
            
            # Store CV text in for_user_prompt
            for_user_prompt['cv_text'] = cv_text
            status.value = "<b>Status:</b> CV uploaded and processed successfully!"
        except Exception as e:
            status.value = f"<b>Status:</b> Error processing PDF: {str(e)}"

        time.sleep(0.5) # Short pause between upload and submit messages to display both
        
        if for_user_prompt['cv_text']:
            #print("CV Submitted:")
            #print(for_user_prompt['cv_text'])
            status.value = "<b>Status:</b> CV submitted successfully!"
            

# Attach actions to buttons
submit_cv_button.on_click(submit_cv_action)

# Layout
cv_buttons = VBox([submit_cv_button])

# Display all widgets
display(VBox([
    HTML(value="<h2>Import CV and submit</h2>"),
    cv_upload,
    cv_buttons,
    status
]))

In [37]:
# Prepare the user prompt that we will send to open ai (added URL for the context)
user_prompt = f"""
Job Posting: 
{for_user_prompt['job_posting']}

CV: 
{for_user_prompt['cv_text']}

Url:
{for_user_prompt['job_posting_url']}
"""

In [38]:
# Define messages with system_prompt and user_prompt
def messages_for(system_prompt_input, user_prompt_input):
    return [
        {"role": "system", "content": system_prompt_input},
        {"role": "user", "content": user_prompt_input}
    ]

In [39]:
# And now: call the OpenAI API. 
response = openai.chat.completions.create(
    model =  MODEL,
    messages = messages_for(system_prompt, user_prompt)
)

# Response is provided in Markdown and displayed accordingly
display(Markdown(response.choices[0].message.content))

APIConnectionError: Connection error.